# Testing the code

This notebook is used to flesh out simple programming errors when writing code. It therefore focuses on the graph building part of the graph, and not on the testable part. Note that we assume in the code that the graph is connected, i.e that there is a path between any two nodes in the graph. The case of non-connected graphs is more complex but, fortunatly, it does not concern us.


## Creating a toy dataset

In [1]:
from importlib import reload
import sys
import os
import numpy as np
import pandas as pd
import random
sys.path.append(os.path.abspath("/Users/hector/Documents/BGWAS2/COIN/Scripts"))
from COIN import *
import graphExplo
reload(graphExplo)
from graphExplo import *

In [2]:
sys.path.append(os.path.abspath("/Users/hector/Documents/BGWAS2/COIN/Scripts"))
import toyDataset
Pop, neighbours, pattern, Pheno, edges = toyDataset.generateToyDataset()
alpha = .05
Lengths = np.ones(pattern.shape[0])
G = structure(Pop, neighbours, pattern, Pheno, Lengths)
n1s, n2s = G.ns()
TH = chi2.isf(.05 / 1, 1)

## Test the functions
### Exploring from one node

In [3]:
random.seed(24)
n = int(n1s.sum() + n2s.sum())
nNodes = G.pattern.shape[0]
R = testables(alpha, nNodes, 10 ** 7)
S = graph(n = n)
explored = check_itemtable(0, G.pattern[0,:], R.itemtable)
if explored:
    next
S.add_node(0, G.pattern[0,:], neighbours = G.neighbours[0], length = G.lengths[0])
# Then we go on enumerating all pairs.
# Note: a single node has no Vertex separator.
R.enum(S, G, n1s, n2s, verbose = True, Lmax = 500)

### Testing itemtable

In [4]:
explored = check_itemtable(0, G.pattern[0,:], R.itemtable)
assert(explored)
explored = check_itemtable(R.subgraphs[2].squareNodes[0].label,
                           G.pattern[int(R.subgraphs[2].squareNodes[0].label),:],
                           R.itemtable)
assert(explored)
explored = check_itemtable(0, G.pattern[0,:], R.itemtable)
assert(explored)
explored = check_itemtable(R.subgraphs[2].squareNodes[431].label,
                           R.subgraphs[2].ys,
                           R.itemtable)
assert(explored)

### Checking the pruning

In [5]:
node = 123
R = testables(alpha, nNodes, 10 ** 7)
G.pattern[node] = np.ones(G.pattern[node].shape)
S = graph(n = n)
explored = check_itemtable(node, G.pattern[node,:], R.itemtable)
if explored:
    next
S.add_node(node, G.pattern[node,:], neighbours = G.neighbours[node], length = G.lengths[node])
# Then we go on enumerating all pairs.
# Note: a single node has no Vertex separator.
R.enum(S, G, n1s, n2s, verbose = True, Lmax = 500)
assert(all(R.subgraphs == None))

In [6]:
R = testables(alpha, nNodes, 10 ** 7)
S = graph(n = n)
explored = check_itemtable(0, G.pattern[0,:], R.itemtable)
if explored:
    next
S.add_node(0, G.pattern[0,:], neighbours = G.neighbours[0], length = G.lengths[0])
# Then we go on enumerating all pairs.
# Note: a single node has no Vertex separator.
R.enum(S, G, n1s, n2s, verbose = True, Lmax = 500)
S = R.subgraphs[0]
S.Env
S.prunable
S.frequencies(G.Pop)
assert(S.Env == envelope(S.frequencies(G.Pop), n1s, n2s))